In [1]:
import numpy as np
import copy
import math
from PIL import Image

# (1)LookAt函数实现

In [2]:
def LookAt(eye, center, up): #构建视图矩阵，
    forward = center - eye
    forward = forward.astype(float)  # 显式将 forward 向量的数据类型设置为浮点数
    forward /= np.linalg.norm(forward)

    right = np.cross(forward, up)
    right /= np.linalg.norm(right)

    new_up = np.cross(forward, right)#防止给出的up与forward方向不垂直，保证为直角系

    # 构建视图矩阵
    view_matrix = np.eye(4)
    view_matrix[0, :3] = right
    view_matrix[1, :3] = new_up
    view_matrix[2, :3] = -forward
    view_matrix[:3, 3] = -np.dot([right, new_up, -forward], eye)#将eye的世界坐标通过点积计算得到在相机坐标系的位置，并放在第四列
    return view_matrix

# (3)Ortho函数实现

In [3]:
def Ortho(left, right, bottom, top, near, far): #正交投影
    Ortho_matrix = np.array([
    [2 / (right - left), 0, 0, -(right + left) / (right - left)],
    [0, 2 / (top - bottom), 0, -(top + bottom) / (top - bottom)],
    [0, 0, -2 / (far - near), (far + near) / (far - near)],#由于z轴方向与forward方向相反
    [0, 0, 0, 1]
])
    return Ortho_matrix

# (2)Perspective函数实现

In [4]:
def Perspective(fov, aspect=1.0, near=0.1, far=10.0): #透视投影
    near , far = far , near# 这里near比far小，为了保持右手系一贯性交换near、far
    mat = np.array([
        [near, 0, 0, 0],
        [0, near, 0, 0],
        [0, 0, (far + near), -(far * near)],   
        [0, 0, 1, 0]
    ]) 
    H = 2 * np.tan(np.radians(fov)/2) * np.abs(near)
    W = H * aspect
    top = H/2
    bottom = -H/2
    left = -W/2
    right = W/2
    Ortho_matrix = Ortho(left, right, bottom, top, near, far)

    Perspective_matrix = Ortho_matrix @ mat
    return Perspective_matrix

# (4)(5)Triangle类补全，包括了inside函数与rotate_norm函数

In [6]:
class Triangle(object): 
    
    def __init__(self):
        
        self.vertices = np.zeros((3, 3))
        self.normals = np.eye(3)
        self.colors = np.zeros((3,3))
        self.tex_coords = np.zeros((3, 2))
    
    def setVertex(self, ind, x, y, z):
        self.vertices[ind][0] = x
        self.vertices[ind][1] = y
        self.vertices[ind][2] = z
    
    def setColor(self, ind, r, g, b):
        self.colors[ind][0] = r
        self.colors[ind][1] = g
        self.colors[ind][2] = b
    
    def setNormal(self, ind, nx, ny, nz):
        self.normals[ind][0] = x
        self.normals[ind][1] = y
        self.normals[ind][2] = z

    def barycentric_coordinates(self, x, y, z):
        A = self.vertices[0]
        B = self.vertices[1]
        C = self.vertices[2]

        detT = (B[1] - C[1]) * (A[0] - C[0]) + (C[0] - B[0]) * (A[1] - C[1])
        alpha = ((B[1] - C[1]) * (x - C[0]) + (C[0] - B[0]) * (y - C[1])) / detT
        beta = ((C[1] - A[1]) * (x - C[0]) + (A[0] - C[0]) * (y - C[1])) / detT
        gamma = 1 - alpha - beta
        return (alpha, beta, gamma)
    
    def setTexCoord(self, ind, tx, ty):
        self.tex_coords[ind][0] = tx
        self.tex_coords[ind][1] = ty

    def rotate_norm(self, thea):
        
        A = self.vertices[0]
        B = self.vertices[1]
        C = self.vertices[2]

        edge1 = B - A
        edge2 = C - A
        normal = np.cross(edge1, edge2)

        center = (A + B + C) / 3.0

        cos_thea = np.cos(thea)
        sin_thea = -np.sin(thea)
        rotation_matrix = np.array([
            [cos_thea + (1 - cos_thea) * normal[0]**2, (1 - cos_thea) * normal[0] * normal[1] - sin_thea * normal[2], (1 - cos_thea) * normal[0] * normal[2] + sin_thea * normal[1]],
            [(1 - cos_thea) * normal[0] * normal[1] + sin_thea * normal[2], cos_thea + (1 - cos_thea) * normal[1]**2, (1 - cos_thea) * normal[1] * normal[2] - sin_thea * normal[0]],
            [(1 - cos_thea) * normal[0] * normal[2] - sin_thea * normal[1], (1 - cos_thea) * normal[1] * normal[2] + sin_thea * normal[0], cos_thea + (1 - cos_thea) * normal[2]**2]
        ])# 创建旋转矩阵
        self.vertices = (rotation_matrix @ (self.vertices - center).T).T + center

    def rotate_zyx(self, angle_z, angle_y, angle_x,time):

        angle_z = np.radians(angle_z)*time  #根据time设定旋转角
        # 创建绕Z轴的旋转矩阵
        rotation_matrix_z = np.array([
            [np.cos(angle_z), -np.sin(angle_z), 0],
            [np.sin(angle_z), np.cos(angle_z), 0],
            [0, 0, 1]
        ])
        self.vertices = (rotation_matrix_z @ self.vertices.T).T#绕z轴旋转

        angle_y = np.radians(angle_y)*time  
        # 创建绕Y轴的旋转矩阵
        rotation_matrix_y = np.array([
            [np.cos(angle_y), 0, np.sin(angle_y)],
            [0, 1, 0],
            [-np.sin(angle_y), 0, np.cos(angle_y)]
        ])
        self.vertices = (rotation_matrix_y @ self.vertices.T).T

        angle_x = np.radians(angle_x)*time  
        # 创建绕X轴的旋转矩阵
        rotation_matrix_x = np.array([
            [1, 0, 0],
            [0, np.cos(angle_x), -np.sin(angle_x)],
            [0, np.sin(angle_x), np.cos(angle_x)]
        ])
        self.vertices = (rotation_matrix_x @ self.vertices.T).T

        #根据时间设定颜色
        self.setColor(0, 1-time, time, 0)
        self.setColor(1, 1-time, time, 0)
        self.setColor(2, 1-time, time, 0)

    def rotate_mat(self, mat): #mat 3x3
        self.vertices = (mat @ self.vertices.T).T
        
        
    
    def inside(self, x, y, z): #判断点(x,y,z)是否在三角形里
        A = self.vertices[0]
        B = self.vertices[1]
        C = self.vertices[2]

        # 计算点到三个顶点的向量
        PA = A - np.array([x, y, z])
        PB = B - np.array([x, y, z])
        PC = C - np.array([x, y, z])

        # 计算三角形的三个边的法线向量
        AB = B - A
        BC = C - B
        CA = A - C

        # 计算点到三个顶点的法线向量
        normal1 = np.cross(AB, PA)
        normal2 = np.cross(BC, PB)
        normal3 = np.cross(CA, PC)

        # 检查点是否在三角形内
        if (np.dot(normal1, normal2) > 0) and (np.dot(normal2, normal3) > 0):
            return True
        else:
            return False
        
    def to_homogeneous_coordinates(self):
        return np.hstack((self.vertices, np.ones((3, 1))))

# 光栅化补全，实现了颜色插值与深度测试

In [7]:
class Rasterization(object):

    def __init__(self, width, height):
        #max = 1000000
        self.color_buf = np.zeros((height, width, 3))
        self.depth_buf = np.zeros((height, width, 1))
        #self.depth_buf[:, :, 0] = max
        self.view_m = np.eye(4) #视图矩阵
        self.proj_m = np.eye(4) #投影矩阵

    def setViewM(self, mat):
        self.view_m = mat

    def setProjM(self, mat):
        self.proj_m = mat
    
    def rasterize_triangle(self, t): #光栅化一个三角形

        H, W, _ = self.color_buf.shape

        v4 = t.to_homogeneous_coordinates()
        v4 = (self.proj_m @ self.view_m @ v4.T).T
        v4 = v4 / np.repeat(v4[:,3:], 4, axis=1)

        v4 = v4 * 0.5 + 0.5

        raster_t = Triangle()
        raster_t.setVertex(0, v4[0][0]*W, v4[0][1]*H, 0)
        raster_t.setVertex(1, v4[1][0]*W, v4[1][1]*H, 0)
        raster_t.setVertex(2, v4[2][0]*W, v4[2][1]*H, 0)
        
        raster_tz = Triangle()
        raster_tz.setVertex(0, v4[0][0] * W, v4[0][1] * H, v4[0][2])
        raster_tz.setVertex(1, v4[1][0] * W, v4[1][1] * H, v4[1][2])
        raster_tz.setVertex(2, v4[2][0] * W, v4[2][1] * H, v4[2][2])

        for x in range(W):
            for y in range(H):
                if raster_t.inside(x, H-1-y, 0):
                    barycentric = raster_t.barycentric_coordinates(x, H - 1 - y, 0)                   
                    current_depth = (barycentric[0] * v4[0][2] + barycentric[1] *v4[1][2] + barycentric[2] * v4[2][2])
                    if current_depth > self.depth_buf[y][x]:
                        # 如果当前深度小于深度缓冲区中的值，更新深度缓冲区
                        color = (barycentric[0] * t.colors[0] + barycentric[1] * t.colors[1] + barycentric[2] * t.colors[2])
                        self.depth_buf[y][x] = current_depth
                        self.color_buf[y][x] = color
        
    
    def render(self, t_list): #渲染，输入一个三角形的list

        for t in t_list:
            self.rasterize_triangle(t)

# 数据写入执行，可以在此处调用旋转操作

In [8]:
R = Rasterization(256, 256)
R.setViewM(LookAt(np.array([0.0, 0.0, 3.0]), np.array([0.0, 0.0, 0.0]), np.array([0.0, 1.0, 0.0])))
R.setProjM(Perspective(60))

t = Triangle()
t.setVertex(0, 0, 1.0, 0.0)
t.setVertex(1, -1.0, 0.0, 1.0)
t.setVertex(2, 1.0, 0.0, 1.0)

t.setColor(0, 1.0, 0.0, 0.0)
t.setColor(1, 0.0, 1.0, 0.0)
t.setColor(2, 0.0, 0.0, 1.0)

T1 = Triangle()
T1.setVertex(0, 0.0, 2.0, 0.0)
T1.setVertex(1, -2.0, 0.0, 0.0)
T1.setVertex(2, 0.0, 0.0, 0.0)

T1.setColor(0, 1.0, 0.0, 0.0)
T1.setColor(1, 1.0, 0.0, 0.0)
T1.setColor(2, 1.0, 0.0, 0.0)

T2 = Triangle()
T2.setVertex(0, -0.5, 0.5, 0.5)
T2.setVertex(1, 1.5, 0.0, 0.5)
T2.setVertex(2, 2.0, 1.0, 0.5)

T2.setColor(0, 0.0, 1.0, 0.0)
T2.setColor(1, 0.0, 1.0, 0.0)
T2.setColor(2, 0.0, 1.0, 0.0)

T = Triangle()
T.setVertex(0, 0.0, 0.0, 0.0)
T.setVertex(1, 2.0, 0.0, 0.0)
T.setVertex(2, 1.0, 1.0, 0.0)

T.setColor(0, 1.0, 0.0, 0.0)
T.setColor(1, 1.0, 0.0, 0.0)
T.setColor(2, 1.0, 0.0, 0.0)

#t.rotate_norm(np.pi*3/4)
T.rotate_zyx(60,90,60,0)
R.render([T])

Image.fromarray((R.color_buf*255).astype("uint8")).show()
#Image.fromarray((R.color_buf*255).astype("uint8")).save('depthtest.jpg')